In [1082]:
import numpy as np
import pandas as pd
import itertools
from itertools import combinations
import calendar
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit
from collections import Counter
import math
from sklearn.model_selection import PredefinedSplit

In [1083]:
#momentum factor (monthly)
ff = pd.read_csv("F-F_Momentum_Factor.csv")

#monthly returns on VWM
vwm = pd.read_csv("VWM Monthly Returns.csv")

# the 1, 5, and 10-year constant maturity series from FRED monthly
one = pd.read_csv("GS1.csv")
five = pd.read_csv("GS5.csv")
ten = pd.read_csv("GS10.csv")

#AAA and BAA monthly
AAA = pd.read_csv("AAA.csv")
BAA = pd.read_csv("BAA.csv")

#CPI monthly
CPI = pd.read_csv("CPILFESL.csv")

#Unemployment rate monthly
UR = pd.read_csv("UNRATE.csv")

#Industrial Productivity monthly
INDPRO = pd.read_csv("INDPRO.csv")

# Display the first few rows of the DataFrame
print(ff.head())
print(vwm.head())
print(one.head())
print(five.head())
print(ten.head())
print(AAA.head())
print(BAA.head())
print(CPI.head())
print(UR.head())
print(INDPRO.head())




   Unnamed: 0  Mom   
0      195701   -3.09
1      195702    0.32
2      195703   -0.18
3      195704    1.00
4      195705    0.48
   Unnamed: 0  Mkt-RF   SMB   HML    RF    Rm
0      195701   -3.58  3.37  2.81  0.27 -3.31
1      195702   -2.06 -0.72 -0.72  0.24 -1.82
2      195703    2.13  0.26 -0.48  0.23  2.36
3      195704    4.26 -1.59 -1.45  0.25  4.51
4      195705    3.45 -1.05 -2.10  0.26  3.71
  observation_date   GS1
0       1957-01-01  3.37
1       1957-02-01  3.38
2       1957-03-01  3.42
3       1957-04-01  3.49
4       1957-05-01  3.48
  observation_date   GS5
0       1957-01-01  3.47
1       1957-02-01  3.39
2       1957-03-01  3.46
3       1957-04-01  3.53
4       1957-05-01  3.64
  observation_date  GS10
0       1957-01-01  3.46
1       1957-02-01  3.34
2       1957-03-01  3.41
3       1957-04-01  3.48
4       1957-05-01  3.60
  observation_date   AAA
0       1957-01-01  3.77
1       1957-02-01  3.67
2       1957-03-01  3.66
3       1957-04-01  3.67
4       1957-05-0

In [1084]:
#Variable Construction

#TERM - The difference between the 10-year and the 1-year (Y10-Y1)

TERM = ten['GS10']-one['GS1']

#CURVE - The 10-year yield plus the 1-year yield minus 2 times the 5-year yield Y10 −2Y5 +Y1 = (Y10 −Y5)−(Y5 −Y1).

CURVE = ten['GS10'] + one['GS1'] - 2*five['GS5']

# DEFAULT - The difference between the AAA and the BAA yields YAAA-YBAA
DEFAULT = AAA['AAA']-BAA['BAA']



In [1085]:
CPI.set_index('observation_date', inplace=True)


In [1086]:
# INFLATION - The year-over-year difference of log CPI: lnCPIt −lnCPIt−12.
# Step 1: Compute log(CPI)
CPI['log_CPI'] = np.log(CPI['CPILFESL'])
# Step 2: Shift log(CPI) by 12 months
CPI['log_CPI_lag_12'] = CPI['log_CPI'].shift(12)
# Step 3: Compute year-over-year difference
CPI['INFLATION'] = CPI['log_CPI'] - CPI['log_CPI_lag_12']
print(CPI)

                  CPILFESL   log_CPI  log_CPI_lag_12  INFLATION
observation_date                                               
1957-01-01          28.500  3.349904             NaN        NaN
1957-02-01          28.600  3.353407             NaN        NaN
1957-03-01          28.700  3.356897             NaN        NaN
1957-04-01          28.800  3.360375             NaN        NaN
1957-05-01          28.800  3.360375             NaN        NaN
...                    ...       ...             ...        ...
2024-06-01         318.346  5.763139        5.730895   0.032244
2024-07-01         318.872  5.764790        5.733163   0.031627
2024-08-01         319.768  5.767596        5.735462   0.032134
2024-09-01         320.767  5.770715        5.738648   0.032067
2024-10-01         321.666  5.773514        5.741046   0.032468

[814 rows x 4 columns]


In [1087]:
CPI.reset_index(inplace=True)
 

In [1088]:
CPI

,observation_date,CPILFESL,log_CPI,log_CPI_lag_12,INFLATION
0,1957-01-01,28.500,3.349904,NaN,NaN
1,1957-02-01,28.600,3.353407,NaN,NaN
2,1957-03-01,28.700,3.356897,NaN,NaN
3,1957-04-01,28.800,3.360375,NaN,NaN
4,1957-05-01,28.800,3.360375,NaN,NaN
...,...,...,...,...,...
809,2024-06-01,318.346,5.763139,5.730895,0.032244
810,2024-07-01,318.872,5.764790,5.733163,0.031627
811,2024-08-01,319.768,5.767596,5.735462,0.032134
812,2024-09-01,320.767,5.770715,5.738648,0.032067


In [1089]:
INFLATION = CPI['INFLATION']

In [1090]:
print(INFLATION.head(13))


0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
5          NaN
6          NaN
7          NaN
8          NaN
9          NaN
10         NaN
11         NaN
12    0.027683
Name: INFLATION, dtype: float64


ANALYSIS

In [1091]:
#number of rows of each dataframe
INFLATION.shape




(814,)

In [1092]:
#Split the data into in sample and out of sample

#have to start from 12 because INFLATION has no numbers for the first 12 columns

#814-12 = 802, 802/2 = 401 obs in each split of data, so start from 12 go to 12+401 = 

#in sample _train, out of sample _test
vwm_train = vwm['Rm'][12:413]
vwm_test = vwm['Rm'][413:]

ff_train = ff[12:413]
ff_test = ff[413:]

TERM_train = TERM[12:413]
TERM_test = TERM[413:]

CURVE_train = CURVE[12:413]
CURVE_test = CURVE[413:]

DEFAULT_train = DEFAULT[12:413]
DEFAULT_test = DEFAULT[413:]

INFLATION_train = INFLATION[12:413]
INFLATION_test = INFLATION[413:]

UR_train = UR[12:413]
UR_test = UR[413:]

INDPRO_train = INDPRO[12:413]
INDPRO_test = INDPRO[413:]



In [1093]:
vwm_train

12     4.94
13    -1.40
14     3.36
15     3.17
16     2.42
       ... 
408    5.21
409    7.67
410    3.09
411    0.25
412    4.12
Name: Rm, Length: 401, dtype: float64

# Question 1

# KITCHEN SINK MODEL in sample (all available data)

## FOR VWM 

In [1094]:
import statsmodels.api as sm
from IPython.display import HTML

In [1095]:
#construct dataframe containing all regressors
factors = pd.DataFrame()
factors['Term'] = TERM[12:]
factors['Curve'] = CURVE[12:]
factors['Default'] = DEFAULT[12:]
factors['Inflation'] = INFLATION[12:]
factors['Unemployment'] = UR['UNRATE'][12:]
factors['Industrial Productivity'] = INDPRO['INDPRO'][12:]

In [1096]:
vwm

,Unnamed: 0,Mkt-RF,SMB,HML,RF,Rm
0,195701,-3.58,3.37,2.81,0.27,-3.31
1,195702,-2.06,-0.72,-0.72,0.24,-1.82
2,195703,2.13,0.26,-0.48,0.23,2.36
3,195704,4.26,-1.59,-1.45,0.25,4.51
4,195705,3.45,-1.05,-2.10,0.26,3.71
...,...,...,...,...,...,...
809,202406,2.77,-3.06,-3.31,0.41,3.18
810,202407,1.24,6.80,5.74,0.45,1.69
811,202408,1.61,-3.55,-1.13,0.48,2.09
812,202409,1.74,-0.17,-2.59,0.40,2.14


In [1097]:
vwm_kitchen_sink = vwm['Rm'][12:]


In [1098]:
factors_w_constant = sm.add_constant(factors)
factors_w_constant.head()

,const,Term,Curve,Default,Inflation,Unemployment,Industrial Productivity
12,1.0,0.44,-0.02,-1.23,0.027683,5.8,20.1606
13,1.0,1.06,-0.52,-1.07,0.027588,6.4,19.7305
14,1.0,1.14,-0.46,-1.05,0.027493,6.7,19.4885
15,1.0,1.43,-0.59,-1.07,0.024015,7.4,19.1660
16,1.0,1.55,-0.53,-1.05,0.024015,7.4,19.3541


In [1099]:
heter_results_vwm = {}
homosk_results_vwm = {}

mod = sm.OLS(vwm_kitchen_sink, factors_w_constant)
heter_results_vwm = mod.fit(cov_type="HC0")
homosk_results_vwm = sm.OLS(vwm_kitchen_sink, factors_w_constant).fit()


In [1100]:
heter_results_vwm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     Rm   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     2.223
Date:                Mon, 13 Jan 2025   Prob (F-statistic):             0.0391
Time:                        17:59:41   Log-Likelihood:                -2318.9
No. Observations:                 802   AIC:                             4652.
Df Residuals:                     795   BIC:                             4685.
Df Model:                           6                                         
Covariance Type:                  HC0                                         
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                      -0.7570      0.965     -0.784      0.433      -2.649       1.135
Term                       -0.0293      0.352     -0.083      0.934      -0.719       0.660
Curve                       0.2557      0.537      0.476      0.634      -0.797       1.308
Default                     0.0913      0.781      0.117      0.907      -1.438       1.621
Inflation                  -7.7901     10.629     -0.733      0.464     -28.623      13.043
Unemployment                0.4021      0.184      2.190      0.028       0.042       0.762
Industrial Productivity    -0.0026      0.008     -0.330      0.741      -0.018       0.013
==============================================================================
Omnibus:                       64.074   Durbin-Watson:                   1.933
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              127.705
Skew:                          -0.503   Prob(JB):                     1.86e-28
Kurtosis:                       4.676   Cond. No.                     4.18e+03
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 4.18e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## FOR FF

In [1101]:
ff_kitchen_sink = ff['Mom   '][12:]

In [1102]:
heter_results_ff = {}
homosk_results_ff = {}

mod = sm.OLS(ff_kitchen_sink, factors_w_constant)
heter_results_ff = mod.fit(cov_type="HC0")
homosk_results_ff = sm.OLS(ff_kitchen_sink, factors_w_constant).fit()


In [1103]:
heter_results_ff.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Mom      R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     1.208
Date:                Mon, 13 Jan 2025   Prob (F-statistic):              0.300
Time:                        17:59:41   Log-Likelihood:                -2256.9
No. Observations:                 802   AIC:                             4528.
Df Residuals:                     795   BIC:                             4561.
Df Model:                           6                                         
Covariance Type:                  HC0                                         
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                       1.0814      0.806      1.341      0.180      -0.499       2.662
Term                       -0.2937      0.332     -0.884      0.377      -0.945       0.358
Curve                      -0.7099      0.549     -1.292      0.196      -1.787       0.367
Default                     1.6530      0.935      1.768      0.077      -0.179       3.485
Inflation                  17.2208     10.375      1.660      0.097      -3.114      37.556
Unemployment                0.0569      0.160      0.357      0.721      -0.256       0.370
Industrial Productivity     0.0044      0.008      0.572      0.567      -0.011       0.019
==============================================================================
Omnibus:                      225.479   Durbin-Watson:                   1.973
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2146.236
Skew:                          -0.984   Prob(JB):                         0.00
Kurtosis:                      10.769   Cond. No.                     4.18e+03
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 4.18e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

LEFT TO DO
- model selection using either forward or backward selection 
- pick best model for both VWM and ff
- use the best model to predict in sample
- choose return threshold and weights for risky and risk free asset
- construct portfolios with risky and risk free asset based on this weight function
- compute sharpe ratios for these portfolios and compare it to the sharpe ratio of the original portolio??

- repeat step 3 and below for out of sample

# UNIVARIATE Regression

## VWM

In [1104]:
factor_names = [col for col in factors_w_constant.columns]

# Run univariate regressions for each factor
homosk_regression_results = {}
heter_regression_results = {}

for factor in factor_names:
    X = sm.add_constant(factors_w_constant[factor])  # Add constant term
    y = vwm_kitchen_sink
    # Perform regression
    model = sm.OLS(y, X)

    # Store results
    homosk_regression_results[f"{factor}"] = model.fit()
    heter_regression_results[f"{factor}"] = model.fit(cov_type="HC0")
        


In [1105]:
for i in heter_regression_results:
    print(heter_regression_results[i].summary())

                            OLS Regression Results                            
Dep. Variable:                     Rm   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 13 Jan 2025   Prob (F-statistic):                nan
Time:                        17:59:41   Log-Likelihood:                -2326.6
No. Observations:                 802   AIC:                             4655.
Df Residuals:                     801   BIC:                             4660.
Df Model:                           0                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9653      0.155      6.210      0.0

In [1106]:
homosk_regression_results

{'const': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x154787110>,
 'Term': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x152224710>,
 'Curve': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x154d2e510>,
 'Default': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x154eb7d10>,
 'Inflation': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x154eb7e90>,
 'Unemployment': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x154eb6780>,
 'Industrial Productivity': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x154eb6450>}

In [1107]:
uni_bic_values = [heter_regression_results['Term'].bic, heter_regression_results['Curve'].bic, heter_regression_results['Default'].bic,
              heter_regression_results['Inflation'].bic, heter_regression_results['Unemployment'].bic, heter_regression_results['Industrial Productivity'].bic]

In [1108]:
uni_bic_values

[4663.876516932659,
 4666.24525023615,
 4664.14020901853,
 4666.442119598632,
 4652.9451573433225,
 4666.25965984063]

In [1109]:
min(uni_bic_values)

4652.9451573433225

In [1110]:
uni_r2_values = [heter_regression_results['Term'].rsquared, heter_regression_results['Curve'].rsquared, heter_regression_results['Default'].rsquared,
              heter_regression_results['Inflation'].rsquared, heter_regression_results['Unemployment'].rsquared, heter_regression_results['Industrial Productivity'].rsquared]

In [1111]:
uni_r2_values

[0.0032756694914771733,
 0.0003274598175899701,
 0.002947899503249918,
 8.203705848797149e-05,
 0.016768979243446447,
 0.0003094984518540178]

In [1112]:
max(uni_r2_values)

0.016768979243446447

# FF Univariate

In [1113]:
factor_names = [col for col in factors_w_constant.columns]

# Run univariate regressions for each factor
ff_homosk_regression_results = {}
ff_heter_regression_results = {}

for factor in factor_names:
    X = sm.add_constant(factors_w_constant[factor])  # Add constant term
    y = ff_kitchen_sink
    # Perform regression
    model = sm.OLS(y, X)

    # Store results
    ff_homosk_regression_results[f"{factor}"] = model.fit()
    ff_heter_regression_results[f"{factor}"] = model.fit(cov_type="HC0")

In [1114]:
for i in ff_heter_regression_results:
    print(ff_heter_regression_results[i].summary())

                            OLS Regression Results                            
Dep. Variable:                 Mom      R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 13 Jan 2025   Prob (F-statistic):                nan
Time:                        17:59:41   Log-Likelihood:                -2267.4
No. Observations:                 802   AIC:                             4537.
Df Residuals:                     801   BIC:                             4542.
Df Model:                           0                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6263      0.144      4.337      0.0

In [1115]:
uni_ff_bic_values = [ff_heter_regression_results['Term'].bic, ff_heter_regression_results['Curve'].bic, ff_heter_regression_results['Default'].bic,
              ff_heter_regression_results['Inflation'].bic, ff_heter_regression_results['Unemployment'].bic, 
              ff_heter_regression_results['Industrial Productivity'].bic]

In [1116]:
uni_ff_bic_values

[4545.852678821095,
 4548.145149385899,
 4537.198404436396,
 4547.313878123494,
 4544.175565898075,
 4546.628251103527]

In [1117]:
min(uni_ff_bic_values)

4537.198404436396

In [1118]:
ff_uni_r2_values = [ff_heter_regression_results['Term'].rsquared, ff_heter_regression_results['Curve'].rsquared, 
                    ff_heter_regression_results['Default'].rsquared, ff_heter_regression_results['Inflation'].rsquared, 
                    ff_heter_regression_results['Unemployment'].rsquared, ff_heter_regression_results['Industrial Productivity'].rsquared]

In [1119]:
ff_uni_r2_values

[0.0029722412992705394,
 0.00011821809141177031,
 0.013673193949298335,
 0.0011540564771821415,
 0.005055010638834845,
 0.002007601513593249]

In [1120]:
max(ff_uni_r2_values)

0.013673193949298335

# Model Selection for Whole Sample

In [1121]:
# model selection using backward stepwise

# variables are term, curve, default, inflation, unemployment, productivity

# VWM

In [1122]:
from collections import defaultdict
import pandas as pd
import statsmodels.api as sm

# Initialize dictionaries to store results
homosk_results_5 = {}
heter_results_5 = {}

# Copy factors and initialize the list of factor names
updated_factors = factors_w_constant.copy()  # Copy to avoid modifying the original
f_names = factors.columns
remaining_predictors = defaultdict(list)  # Dictionary to store remaining predictors at each iteration
bic_values = []  # List to store BIC values of the chosen model in each iteration

for i in range(5):  # Loop 5 times for backward regression
    sse_dict = {}
    models = {}  # Store models for BIC extraction later

    for pred in f_names:
        # Remove one variable
        col = updated_factors.drop(columns=[pred])
        
        # Perform regression
        model = sm.OLS(vwm_kitchen_sink, col) #specific to VWM
        results = model.fit(cov_type="HC0")
        
        # Store results
        heter_results_5[f"{pred}_iter{i}"] = results
        sse_dict[pred] = results.ssr
        models[pred] = results  # Keep track of models

    # Convert results to DataFrame
    sse = pd.DataFrame(list(sse_dict.items()), columns=['Predictor', 'SSE'])

    # Find the predictor to remove (minimum SSE)
    pred_to_remove = sse['Predictor'][sse['SSE'].idxmin()]

    # Store BIC of the chosen model
    bic_values.append(models[pred_to_remove].bic)

    # Store remaining predictors for this iteration
    remaining_factors = [f for f in f_names if f != pred_to_remove]
    remaining_predictors[f"Iteration_{i}"] = remaining_factors

    # Update factors by removing the selected predictor
    updated_factors = updated_factors.drop(columns=[pred_to_remove])
    f_names = f_names.drop(pred_to_remove)  # Update the list of factors

# Display remaining predictors and BIC values
for iteration, predictors in remaining_predictors.items():
    print(f"{iteration}: {predictors}")

print("\nBIC values of the chosen model in each iteration:")
print(bic_values)


Iteration_0: ['Curve', 'Default', 'Inflation', 'Unemployment', 'Industrial Productivity']
Iteration_1: ['Curve', 'Inflation', 'Unemployment', 'Industrial Productivity']
Iteration_2: ['Curve', 'Inflation', 'Unemployment']
Iteration_3: ['Inflation', 'Unemployment']
Iteration_4: ['Unemployment']

BIC values of the chosen model in each iteration:
[4677.880036385706, 4671.222802971034, 4664.8275083618555, 4658.914947795646, 4652.9451573433225]


In [1123]:
#loop through five variables to one variable (5 iter)
# within each: run iter to remove one variable each time
# fit the new predictors to a return
# store aic, bic, and r2
# compare across all sets of predictors and remove a variable that minimise sse
# store new set of variables for lo, med, and hi

from collections import defaultdict
import pandas as pd
import statsmodels.api as sm

# Initialize dictionaries to store results
homosk_results_5 = {}
heter_results_5 = {}

# Copy factors and initialize the list of factor names
updated_factors = factors_w_constant.copy()  # Copy to avoid modifying the original
f_names = factors.columns
remaining_predictors = defaultdict(list)  # Dictionary to store remaining predictors at each iteration

for i in range(5):  # Loop 5 times for backward regression
    sse_dict = {}

    for pred in f_names:
        # Remove one variable
        col = updated_factors.drop(columns=[pred])
        
        # Perform regression
        model = sm.OLS(vwm_kitchen_sink, col)
        results = model.fit(cov_type="HC0")
        
        # Store results
        heter_results_5[f"{pred}_iter{i}"] = results
        sse_dict[pred] = results.ssr

    # Convert results to DataFrame
    sse = pd.DataFrame(list(sse_dict.items()), columns=['Predictor', 'SSE'])

    # Find the predictor to remove (minimum SSE)
    pred_to_remove = sse['Predictor'][sse['SSE'].idxmin()]

    # Store remaining predictors for this iteration
    remaining_factors = [f for f in f_names if f != pred_to_remove]
    remaining_predictors[f"Iteration_{i}"] = remaining_factors

    # Update factors by removing the selected predictor
    updated_factors = updated_factors.drop(columns=[pred_to_remove])
    f_names = f_names.drop(pred_to_remove)  # Update the list of factors



In [1124]:
bic_values = [heter_results_5['Term_iter0'].bic, heter_results_5['Default_iter1'].bic, heter_results_5['Industrial Productivity_iter2'].bic,
              heter_results_5['Curve_iter3'].bic,heter_results_5['Inflation_iter4'].bic]

In [1125]:
bic_values

[4677.880036385706,
 4671.222802971034,
 4664.8275083618555,
 4658.914947795646,
 4652.9451573433225]

In [1126]:
aic_values = [heter_results_5['Term_iter0'].aic, heter_results_5['Default_iter1'].aic, heter_results_5['Industrial Productivity_iter2'].aic,
              heter_results_5['Curve_iter3'].aic,heter_results_5['Inflation_iter4'].aic]

In [1127]:
rsquared_values = [heter_results_5['Term_iter0'].rsquared, heter_results_5['Default_iter1'].rsquared, heter_results_5['Industrial Productivity_iter2'].rsquared,
              heter_results_5['Curve_iter3'].rsquared,heter_results_5['Inflation_iter4'].rsquared]

In [1128]:
rsquared_values

[0.018989838759107425,
 0.01895329460203632,
 0.01859626810389814,
 0.01764799937653183,
 0.016768979243446447]

In [1129]:
max(rsquared_values)

0.018989838759107425

r^2 of the 3rd iteration is the highest, so the model selected by criteria of r^2 using backward stepwise regression is one that includes 

Iteration_2: ['Curve', 'Inflation', 'Unemployment']


In [1130]:
# check that model selected based on BIC and SSE is the same

sse_values = [heter_results_5['Term_iter0'].ssr, heter_results_5['Default_iter1'].ssr, heter_results_5['Industrial Productivity_iter2'].ssr,
              heter_results_5['Curve_iter3'].ssr,heter_results_5['Inflation_iter4'].ssr]

In [1131]:
sse_values

[15244.459776392863,
 15245.02765627275,
 15250.575688602088,
 15265.311361118733,
 15278.970941407391]

In [1132]:
min(sse_values)

15244.459776392863

In [1133]:
#calculate BIC for each iteration and pick the model with the lowest

# Display remaining predictors and BIC values

for iteration, predictors in remaining_predictors.items():
    print(f"{iteration}: {predictors}")

print("\nBIC values of the chosen model in each iteration:")
print(bic_values)

Iteration_0: ['Curve', 'Default', 'Inflation', 'Unemployment', 'Industrial Productivity']
Iteration_1: ['Curve', 'Inflation', 'Unemployment', 'Industrial Productivity']
Iteration_2: ['Curve', 'Inflation', 'Unemployment']
Iteration_3: ['Inflation', 'Unemployment']
Iteration_4: ['Unemployment']

BIC values of the chosen model in each iteration:
[4677.880036385706, 4671.222802971034, 4664.8275083618555, 4658.914947795646, 4652.9451573433225]


In [1134]:
#first iteration dropped term

In [1135]:
# pick the model with the lowest BIC

# Find the index of the iteration with the lowest BIC
min_bic_index = bic_values.index(min(bic_values))

# Retrieve the corresponding iteration name and predictors
lowest_bic_iteration = f"Iteration_{min_bic_index}"
lowest_bic_predictors = remaining_predictors[lowest_bic_iteration]

# Print results
print(f"The iteration with the lowest BIC is: {lowest_bic_iteration}")
print(f"Predictors in the model with the lowest BIC: {lowest_bic_predictors}")
print(f"Lowest BIC value: {min(bic_values)}")

The iteration with the lowest BIC is: Iteration_4
Predictors in the model with the lowest BIC: ['Unemployment']
Lowest BIC value: 4652.9451573433225


# FF Model Selection


In [1136]:
# Initialize dictionaries to store results
ff_heter_results_5 = {}

# Copy factors and initialize the list of factor names
updated_factors = factors_w_constant.copy()  # Copy to avoid modifying the original
f_names = factors.columns
remaining_predictors = defaultdict(list)  # Dictionary to store remaining predictors at each iteration
ff_model_selection_bic_values = []  # List to store BIC values of the chosen model in each iteration

for i in range(5):  # Loop 5 times for backward regression
    sse_dict = {}
    models = {}  # Store models for BIC extraction later

    for pred in f_names:
        # Remove one variable
        col = updated_factors.drop(columns=[pred])
        
        # Perform regression
        model = sm.OLS(ff_kitchen_sink, col) #specific to ff
        results = model.fit(cov_type="HC0")
        
        # Store results
        ff_heter_results_5[f"{pred}_iter{i}"] = results
        sse_dict[pred] = results.ssr
        models[pred] = results  # Keep track of models

    # Convert results to DataFrame
    sse = pd.DataFrame(list(sse_dict.items()), columns=['Predictor', 'SSE'])

    # Find the predictor to remove (minimum SSE)
    pred_to_remove = sse['Predictor'][sse['SSE'].idxmin()]

    # Store BIC of the chosen model
    ff_model_selection_bic_values.append(models[pred_to_remove].bic)

    # Store remaining predictors for this iteration
    remaining_factors = [f for f in f_names if f != pred_to_remove]
    remaining_predictors[f"Iteration_{i}"] = remaining_factors

    # Update factors by removing the selected predictor
    updated_factors = updated_factors.drop(columns=[pred_to_remove])
    f_names = f_names.drop(pred_to_remove)  # Update the list of factors

# Display remaining predictors and BIC values
for iteration, predictors in remaining_predictors.items():
    print(f"{iteration}: {predictors}")

print("\nBIC values of the chosen model in each iteration:")
print(ff_model_selection_bic_values)


Iteration_0: ['Term', 'Curve', 'Default', 'Inflation', 'Industrial Productivity']
Iteration_1: ['Term', 'Curve', 'Default', 'Inflation']
Iteration_2: ['Curve', 'Default', 'Inflation']
Iteration_3: ['Default', 'Inflation']
Iteration_4: ['Default']

BIC values of the chosen model in each iteration:
[4554.125598202757, 4547.676233837324, 4541.632529091882, 4536.510903294634, 4537.198404436396]


In [1137]:
# pick the model with the lowest BIC

# Find the index of the iteration with the lowest BIC
min_bic_index = ff_model_selection_bic_values.index(min(ff_model_selection_bic_values))

# Retrieve the corresponding iteration name and predictors
lowest_bic_iteration = f"Iteration_{min_bic_index}"
lowest_bic_predictors = remaining_predictors[lowest_bic_iteration]

# Print results
print(f"The iteration with the lowest BIC is: {lowest_bic_iteration}")
print(f"Predictors in the model with the lowest BIC: {lowest_bic_predictors}")
print(f"Lowest BIC value: {min(ff_model_selection_bic_values)}")

The iteration with the lowest BIC is: Iteration_3
Predictors in the model with the lowest BIC: ['Default', 'Inflation']
Lowest BIC value: 4536.510903294634


# Summary Statistics

VWM

VWM Kitchen Sink: R2 = 0.019 for hetero, BIC = 4685

VWM Univariate - Unemployment, R2 = 0.016768979243446447, BIC = 4652.9451573433225

VWM Model Selection - Unemployment, BIC = 4652.9451573433225, don't care about R2 for model selection using backward stepwise because the r^2 of the initial model will always be the highest

FF

FF Kitchen Sink: R2 = 0.026 for hetero, BIC = 4561

FF Univariate - Default, R2 = 0.013673193949298335, BIC = 4537.198404436396

FF Model Selection - iteration with Default and Inflation, BIC = 4536.510903294634

Choose BIC ultimately as the selection criterion becasue we don't wanna overfit and R^2 will always increase never decrease if you add variables

In [1138]:
VWM_BICs = [4685, 4652.9451573433225, 4652.9451573433225]

FF_BICs = [4561, 4537.198404436396, 4536.510903294634]

In [1139]:
min(VWM_BICs)

4652.9451573433225

In [1140]:
min(FF_BICs)

4536.510903294634

SO THE BEST MODEL FOR VWM BASED ON BIC IS THE UNIVARIATE WITH UNEMPLOYMENT, AND THE BEST FOR FF IS THE MODEL SELECTION MODEL WITH INFLATION AND DEFAULT


# Predicting Returns using Best Models and Weighting

# VWM

In [1141]:
pred_vwm = heter_results_5['Inflation_iter4'].predict(sm.add_constant(UR['UNRATE'][12:]))
pred_vwm

12     0.931503
13     1.135403
14     1.237353
15     1.475236
16     1.475236
         ...   
809    0.353787
810    0.421753
811    0.387770
812    0.353787
813    0.353787
Length: 802, dtype: float64

we still just use everything except the first 12 rows because this is the dataset that was used to train the model

In [1142]:
heter_results_5['Inflation_iter4'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     Rm   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     11.79
Date:                Mon, 13 Jan 2025   Prob (F-statistic):           0.000626
Time:                        17:59:41   Log-Likelihood:                -2319.8
No. Observations:                 802   AIC:                             4644.
Df Residuals:                     800   BIC:                             4653.
Df Model:                           1                                         
Covariance Type:                  HC0                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -1.0395      0.600     -1.732      0.083      -2.216       0.137
Unemployment     0.3398      0.099      3.434      0.001       0.146       0.534
==============================================================================
Omnibus:                       67.026   Durbin-Watson:                   1.930
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              130.992
Skew:                          -0.530   Prob(JB):                     3.59e-29
Kurtosis:                       4.672   Cond. No.                         23.0
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [1143]:

# Calculate historical average return (r_bar)
r_bar = vwm_kitchen_sink.mean()

# Apply the weighting function using pandas' `apply` method
weights = pred_vwm.apply(lambda x: 1.5 if x > r_bar else 0.5)

# Combine everything into a DataFrame for clarity
portfolio_weights = pd.DataFrame({
    'Predicted VWM': pred_vwm,
    'Historical Average Return (r_bar)': r_bar,
    'Weights': weights
})

print(portfolio_weights)


     Predicted VWM  Historical Average Return (r_bar)  Weights
12        0.931503                           0.965274      0.5
13        1.135403                           0.965274      1.5
14        1.237353                           0.965274      1.5
15        1.475236                           0.965274      1.5
16        1.475236                           0.965274      1.5
..             ...                                ...      ...
809       0.353787                           0.965274      0.5
810       0.421753                           0.965274      0.5
811       0.387770                           0.965274      0.5
812       0.353787                           0.965274      0.5
813       0.353787                           0.965274      0.5

[802 rows x 3 columns]


In [1144]:
portfolio_returns = pred_vwm * weights + vwm['RF'][12:] * (1-weights)

In [1145]:
new_sharpe = portfolio_returns.mean() / portfolio_returns.std()
new_sharpe

1.2362255107447673

In [1146]:
old_sharpe = vwm_kitchen_sink.mean()/vwm_kitchen_sink.std()
old_sharpe

0.21915316213725497

# FF

In [1147]:
# Combine DEFAULT and INFLATION into a DataFrame
new_data = pd.DataFrame({
    'DEFAULT': DEFAULT[12:].values,
    'INFLATION': INFLATION[12:].values
})

# Add a constant term (intercept)
new_data_with_constant = sm.add_constant(new_data)

# Make predictions
pred_ff = ff_heter_results_5['Curve_iter3'].predict(new_data_with_constant)
print(pred_ff)

0      0.110511
1      0.361138
2      0.390944
3      0.295482
4      0.327029
         ...   
797    1.046099
798    0.987433
799    0.980979
800    0.963987
801    1.065983
Length: 802, dtype: float64


In [1148]:

# Calculate historical average return (r_bar)
r_bar = ff_kitchen_sink.mean()

# Apply the weighting function using pandas' `apply` method
weights = pred_ff.apply(lambda x: 1.5 if x > r_bar else 0.5)

# Combine everything into a DataFrame for clarity
ff_portfolio_weights = pd.DataFrame({
    'Predicted FF': pred_ff,
    'Historical Average Return (r_bar)': r_bar,
    'Weights': weights
})

print(ff_portfolio_weights)


     Predicted FF  Historical Average Return (r_bar)  Weights
0        0.110511                           0.626259      0.5
1        0.361138                           0.626259      0.5
2        0.390944                           0.626259      0.5
3        0.295482                           0.626259      0.5
4        0.327029                           0.626259      0.5
..            ...                                ...      ...
797      1.046099                           0.626259      1.5
798      0.987433                           0.626259      1.5
799      0.980979                           0.626259      1.5
800      0.963987                           0.626259      1.5
801      1.065983                           0.626259      1.5

[802 rows x 3 columns]


In [1149]:
ff_portfolio_returns = pred_ff * weights + vwm['RF'][12:] * (1-weights)

In [1150]:
ff_new_sharpe = ff_portfolio_returns.mean() / ff_portfolio_returns.std()
ff_new_sharpe

1.3301673320661254

In [1151]:
ff_new_sharpe_is = ff_portfolio_returns[413:].mean() / ff_portfolio_returns[413:].std()
ff_new_sharpe_is

1.0546535863579594

In [1152]:
ff_old_sharpe = ff_kitchen_sink.mean()/ff_kitchen_sink.std()
ff_old_sharpe

0.15306401804614264

In [1153]:
ff_old_sharpe_is = ff_kitchen_sink[413:].mean()/ff_kitchen_sink[413:].std()
ff_old_sharpe_is

0.08932453342959115

sharpe ratio is much lower if we only look over the [413:] period so either the mean is a lot lower or SD is a lot higher

# Question 2

# Retrain VWM Model and Compare Sharpe Ratios


In [1154]:
heter_results_oos = {}

pred = sm.add_constant(UR_train['UNRATE'])
        
# Perform regression
model = sm.OLS(vwm_train, pred) #specific to VWM
results = model.fit(cov_type="HC0")

oos_pred = results.predict(sm.add_constant(UR_test['UNRATE']))

In [1155]:
r2_oos_pred = r2_score(vwm_test, oos_pred)

In [1156]:
# Calculate historical average return (r_bar)
r_bar = oos_pred.mean()

# Apply the weighting function using pandas' `apply` method
weights = oos_pred.apply(lambda x: 1.5 if x > r_bar else 0.5)

# Combine everything into a DataFrame for clarity
oos_portfolio_weights = pd.DataFrame({
    'Predicted VWM': oos_pred,
    'Historical Average Return (r_bar)': r_bar,
    'Weights': weights
})

print(oos_portfolio_weights)


     Predicted VWM  Historical Average Return (r_bar)  Weights
413       1.332722                           0.833211      1.5
414       1.290659                           0.833211      1.5
415       1.332722                           0.833211      1.5
416       1.332722                           0.833211      1.5
417       1.374785                           0.833211      1.5
..             ...                                ...      ...
809       0.154957                           0.833211      0.5
810       0.239083                           0.833211      0.5
811       0.197020                           0.833211      0.5
812       0.154957                           0.833211      0.5
813       0.154957                           0.833211      0.5

[401 rows x 3 columns]


In [1157]:
vwm_train

12     4.94
13    -1.40
14     3.36
15     3.17
16     2.42
       ... 
408    5.21
409    7.67
410    3.09
411    0.25
412    4.12
Name: Rm, Length: 401, dtype: float64

In [1158]:
vwm_test

413   -4.52
414    4.73
415    2.78
416   -1.13
417    1.71
       ... 
809    3.18
810    1.69
811    2.09
812    2.14
813   -0.58
Name: Rm, Length: 401, dtype: float64

In [1159]:
oos_portfolio_returns = oos_pred * weights + vwm['RF'][413:] * (1-weights)

#assume the risk free rate for out of sample data is known, only thing we're predicting is portfolio returns out of sample

In [1160]:
oos_new_sharpe = oos_portfolio_returns.mean() / oos_portfolio_returns.std()
oos_new_sharpe

0.9306098659483136

In [1161]:
oos_old_sharpe = vwm_test.mean()/vwm_test.std()
oos_old_sharpe

0.214247306703385

# Retrain FF Model and compare Sharpe Ratios

In [1162]:
#align indices
#aligned_ff_train, aligned_ff_pred = ff_train['Mom   '].align(ff_pred, join='inner', axis=0)

In [1163]:
ff_heter_results_oos = {}

# Combine DEFAULT and INFLATION into a DataFrame
new_data_ff_train = pd.DataFrame({
    'DEFAULT': DEFAULT_train.values,
    'INFLATION': INFLATION_train.values
})

new_data_ff_test = pd.DataFrame({
    'DEFAULT': DEFAULT_test.values,
    'INFLATION': INFLATION_test.values
})

# Add a constant term (intercept)
ff_pred = sm.add_constant(new_data_ff_train)

ff_train_reset = ff_train.reset_index(drop=True)
ff_pred_reset = ff_pred.reset_index(drop=True)
        
# Perform regression
model = sm.OLS(ff_train_reset['Mom   '], ff_pred_reset) 
results = model.fit(cov_type="HC0")

ff_oos_pred = results.predict(sm.add_constant(new_data_ff_test))

In [1164]:
ff_oos_pred

0      0.899154
1      0.932961
2      0.910623
3      0.909239
4      0.863900
         ...   
396    0.968048
397    0.938738
398    0.935105
399    0.926536
400    0.977972
Length: 401, dtype: float64

In [1166]:
r2_ff_oos_pred = r2_score(ff_test['Mom   '], ff_oos_pred)

In [1167]:
# Calculate historical average return (r_bar)
r_bar = ff_oos_pred.mean()

# Apply the weighting function using pandas' `apply` method
weights = ff_oos_pred.apply(lambda x: 1.5 if x > r_bar else 0.5)

# Combine everything into a DataFrame for clarity
ff_oos_portfolio_weights = pd.DataFrame({
    'Predicted VWM': ff_oos_pred,
    'Historical Average Return (r_bar)': r_bar,
    'Weights': weights
})

print(ff_oos_portfolio_weights)

     Predicted VWM  Historical Average Return (r_bar)  Weights
0         0.899154                           0.701713      1.5
1         0.932961                           0.701713      1.5
2         0.910623                           0.701713      1.5
3         0.909239                           0.701713      1.5
4         0.863900                           0.701713      1.5
..             ...                                ...      ...
396       0.968048                           0.701713      1.5
397       0.938738                           0.701713      1.5
398       0.935105                           0.701713      1.5
399       0.926536                           0.701713      1.5
400       0.977972                           0.701713      1.5

[401 rows x 3 columns]


In [1168]:
weights.unique()

array([1.5, 0.5])

In [1169]:
vwm_index_aligned = vwm['RF'][413:].reset_index(drop=True)

ff_oos_portfolio_returns = ff_oos_pred * weights + vwm_index_aligned * (1-weights)

In [1170]:
ff_oos_new_sharpe = ff_oos_portfolio_returns.mean() / ff_oos_portfolio_returns.std()
ff_oos_new_sharpe

1.7181879486637295

In [1171]:
ff_oos_old_sharpe = ff_test['Mom   '].mean()/ff_test['Mom   '].std()
ff_oos_old_sharpe

0.09532709869001367

improvement might be much larger than for in sample because here you only derive the standard deviation over the out of sample period and maybe the returns of ff over the out of sample period were much more variable than in the training period

if we do the same calculation for the in sample data over the same period (observations 413 to end) then a similar trend is observed? -- see the is calculations over the same period above

# Question 3

In [1172]:
from sklearn.metrics import r2_score

In [1178]:
# In sample R2 for VWM

#here we are using the in sample model and predicting the testing data, then calculating the R2 of the model for this range of data
# we're doing this because we can only compare R2 of data happening in the same time range
# so if we want to compare in sample to out of sample we need to calculate the in sample r2 only for the time range of the out of sample data

pred_vwm_is = heter_results_5['Inflation_iter4'].predict(sm.add_constant(UR_test['UNRATE']))

r2_pred_vwm_is = r2_score(vwm_test, pred_vwm_is)

print(r2_pred_vwm_is) #in sample r2 for vwm over same period as oos predictions

print(r2_oos_pred) # out of sample r2 for vwm

0.012653834293923416
0.00955533489570315


In [1179]:
# In sample R2 for FF

#here we are using the in sample model and predicting the testing data, then calculating the R2 of the model for this range of data
# we're doing this because we can only compare R2 of data happening in the same time range
# so if we want to compare in sample to out of sample we need to calculate the in sample r2 only for the time range of the out of sample data

pred_ff_is = ff_heter_results_5['Curve_iter3'].predict(sm.add_constant(new_data_ff_test))

r2_pred_ff_is = r2_score(ff_test['Mom   '], pred_ff_is)
print(r2_pred_ff_is) #in sample r2 for ff
print(r2_ff_oos_pred) # out of sample r2 for ff


0.03150042489292526
0.0177622917635899


question 1

1. univariate for whole sample
2. model selection for whole sample using R^2, AIC, BIC
3. then choose the best univariate, kitchen sink and backward selection model, then select the best one from the 3
4. use the best model to predict returns
5. construct portfolios where returns exceed average historical returns
6. calculate sharpe ratios for each portfolio
7. compare sharpe ratio to original portfolio 

question 2

1. split data in half into train and test
2. train model we selected from question 1 
3. predict returns and calculate out of sample r^2
4. calculate sharpe ratios and compare
5. 

question 3
theoretical question


from collections import defaultdict
import pandas as pd
import statsmodels.api as sm

# Initialize dictionaries to store results
homosk_results_5 = {}
heter_results_5 = {}

# Copy factors and initialize the list of factor names
updated_factors = factors_w_constant.copy()  # Copy to avoid modifying the original
f_names = factors.columns
remaining_predictors = defaultdict(list)  # Dictionary to store remaining predictors at each iteration
bic_values = []  # List to store BIC values of the chosen model in each iteration

for i in range(5):  # Loop 5 times for backward regression
    sse_dict = {}
    models = {}  # Store models for BIC extraction later

    for pred in f_names:
        # Remove one variable
        col = updated_factors.drop(columns=[pred])
        
        # Perform regression
        model = sm.OLS(vwm_kitchen_sink, col) #specific to VWM
        results = model.fit(cov_type="HC0")
        
        # Store results
        heter_results_5[f"{pred}_iter{i}"] = results
        sse_dict[pred] = results.ssr
        models[pred] = results  # Keep track of models

    # Convert results to DataFrame
    sse = pd.DataFrame(list(sse_dict.items()), columns=['Predictor', 'SSE'])

    # Find the predictor to remove (minimum SSE)
    pred_to_remove = sse['Predictor'][sse['SSE'].idxmin()]

    # Store BIC of the chosen model
    bic_values.append(models[pred_to_remove].bic)

    # Store remaining predictors for this iteration
    remaining_factors = [f for f in f_names if f != pred_to_remove]
    remaining_predictors[f"Iteration_{i}"] = remaining_factors

    # Update factors by removing the selected predictor
    updated_factors = updated_factors.drop(columns=[pred_to_remove])
    f_names = f_names.drop(pred_to_remove)  # Update the list of factors

# Display remaining predictors and BIC values
for iteration, predictors in remaining_predictors.items():
    print(f"{iteration}: {predictors}")

print("\nBIC values of the chosen model in each iteration:")
print(bic_values)
